In [1]:
import tensorflow as tf
import numpy as np
import cv2
from matplotlib import pyplot as plt

E:\Anaconda3\envs\tensorflow\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Prerequisites
image_height = 240
image_width = 320
num_bin = 20+1
bin_width = 1.0/(num_bin-1)
num_training_frame = 20

In [3]:
[[[r,c] for c in range(0,4)] for r in range(0,3)]

[[[0, 0], [0, 1], [0, 2], [0, 3]],
 [[1, 0], [1, 1], [1, 2], [1, 3]],
 [[2, 0], [2, 1], [2, 2], [2, 3]]]

In [4]:
model_scope = tf.variable_scope('model', reuse=tf.AUTO_REUSE)
input_scope = tf.variable_scope('input', reuse=tf.AUTO_REUSE)
output_scope = tf.variable_scope('output', reuse=tf.AUTO_REUSE)

In [5]:
with model_scope:
    # Build the model
    fuzzy_histogram = tf.get_variable('fuzzy_histogram',
                                      shape=[image_height, image_width, num_bin],
                                      initializer=tf.constant_initializer(1.0),
                                      trainable=False)
    # Define updating mask
    update_mask = tf.get_variable('update_mask',
                                  shape=[image_height, image_width],
                                  initializer=tf.constant_initializer(1.0),
                                  trainable=False)
    # Intermitent vars
    data_position_in_histogram = tf.get_variable('data_position_in_histogram',
                                                 shape=[image_height, image_width],
                                                 trainable=False)
    pre_index = tf.get_variable('previous_index_in_histogram',
                                shape=[image_height, image_width],
                                dtype=tf.int32,
                                trainable=False)
    pre_weight = tf.get_variable('previous_weight_in_histogram',
                                 shape=[image_height, image_width],
                                 trainable=False)
    next_index = tf.get_variable('next_index_in_histogram',
                                 shape=[image_height, image_width],
                                 dtype=tf.int32,
                                 trainable=False)
    next_weight = tf.get_variable('next_weight_in_histogram',
                                  shape=[image_height, image_width],
                                  trainable=False)
    max_bin_value = tf.get_variable('max_bin_value',
                                    shape=[image_height, image_width],
                                    trainable=False)
    # Intermitent constants
    indexing_constant = tf.constant([[[r,c] for c in range(image_width)] for r in range(image_height)],name='indexing_constant')

In [6]:
with input_scope:
    # Define the input image
    input_image = tf.placeholder(tf.float32, 
                                 shape=[image_height,image_width], 
                                 name='image')
    # Define the iground truth
    label = tf.placeholder(tf.float32, 
                           shape=[image_height,image_width], 
                           name='truth')

In [7]:
with output_scope:
    # Define the raw output 
    raw_segmentation = tf.get_variable('compatibility',
                             shape=[image_height, image_width],
                             trainable=False)
    # Define the synthesis result 
    synthesis_result = tf.get_variable('synthesis',
                             shape=[image_height, image_width],
                             trainable=False)

In [8]:
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES ):
    print(var.name,var.shape)

model/fuzzy_histogram:0 (240, 320, 21)
model/update_mask:0 (240, 320)
model/data_position_in_histogram:0 (240, 320)
model/previous_index_in_histogram:0 (240, 320)
model/previous_weight_in_histogram:0 (240, 320)
model/next_index_in_histogram:0 (240, 320)
model/next_weight_in_histogram:0 (240, 320)
model/max_bin_value:0 (240, 320)
output/compatibility:0 (240, 320)
output/synthesis:0 (240, 320)


In [9]:
print(indexing_constant.name,indexing_constant.shape)

model/indexing_constant:0 (240, 320, 2)


In [10]:
for op in tf.get_default_graph().get_operations():
    if op.type == "Placeholder":
        print(op.name)

input/image
input/truth


In [11]:
# Initialization
def initialize_sess(graph=tf.get_default_graph()):
    sess = tf.Session(graph=graph)
    sess.run(tf.global_variables_initializer())
    return sess

In [12]:
# Define histogram checking
def histogram_checking(sess, gray_float_image):
    '''
    Inputs:
        data_to_check: [height, width], float (from 0.0 to 1.0)
        histogram_to_use: [height, width, num_bins], float
    Outputs:
        result: [height, width], float
    '''
    sess.run(data_position_in_histogram.assign(input_image/bin_width),feed_dict={input_image: gray_float_image})
    sess.run(pre_index.assign(tf.to_int32(tf.floor(data_position_in_histogram))))
    sess.run(pre_weight.assign(data_position_in_histogram-tf.to_float(pre_index)))
    sess.run(next_index.assign(pre_index+1))
    sess.run(next_weight.assign(tf.to_float(next_index)-data_position_in_histogram))
    sess.run(raw_segmentation.assign(tf.add(tf.multiply(tf.gather_nd(fuzzy_histogram,
                                                                     tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                                        pre_weight),
                                            tf.multiply(tf.gather_nd(fuzzy_histogram,
                                                                     tf.concat([indexing_constant, tf.expand_dims(next_index, -1)], -1)),
                                                        next_weight))))

In [13]:
def temp_synthesis(sess):
    # The following can be replaced by avr_pooling
    raw_segmentation_in_np_in_float_scaled_to_255 = 255.0*sess.run(raw_segmentation.value())
    raw_segmentation_in_np_in_uint8 = raw_segmentation_in_np_in_float_scaled_to_255.astype('uint8')
    blurred_mask_uint8 = cv2.medianBlur(raw_segmentation_in_np_in_uint8,9)
    blurred_mask_float = blurred_mask_uint8.astype('float')/256
    sess.run(synthesis_result.assign(blurred_mask_float))

In [14]:
def update_weight_calculation(sess):
    # a*y^5/(x+b) a = 0.0792; b = 0.1585
    sess.run(update_mask.assign(0.0792*synthesis_result**5/(raw_segmentation+0.1585)))

```
def update_histogram(sess):
    sess.run(tf.scatter_update(fuzzy_histogram,
                               indices=tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)],
                               updates=tf.add(tf.multiply(pre_weight,update_mask),
                                              tf.gather_nd(fuzzy_histogram,
                                                           tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1))))))
    sess.run(tf.scatter_update(fuzzy_histogram,
                               indices=tf.concat([indexing_constant, tf.expand_dims(next_index, -1)],
                               updates=tf.add(tf.multiply(next_weight,update_mask),
                                              tf.gather_nd(fuzzy_histogram,
                                                           tf.concat([indexing_constant, tf.expand_dims(next_index, -1)], -1))))))
    sess.run(max_bin_value.assign(tf.reduce_max(fuzzy_histogram,axis=-1)))
    sess.run(fuzzy_histogram.assign(tf.divide(fuzzy_histogram,
                                              tf.tile(tf.expand_dims(max_bin_value, -1),
                                                      [1, 1, num_bin]))))
```

In [15]:
def update_histogram(sess):
    sess.run(fuzzy_histogram.assign_add(tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                                                                                     [image_height*image_width,3]), 
                                                                                  values=tf.reshape(tf.multiply(pre_weight,update_mask),
                                                                                                    [image_height*image_width,]), 
                                                                                  dense_shape=[image_height,image_width,num_bin]))))
    sess.run(fuzzy_histogram.assign_add(tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(next_index, -1)], -1)),
                                                                                                     [image_height*image_width,3]), 
                                                                                  values=tf.reshape(tf.multiply(next_weight,update_mask),
                                                                                                    [image_height*image_width,]), 
                                                                                  dense_shape=[image_height,image_width,num_bin]))))
    sess.run(max_bin_value.assign(tf.reduce_max(fuzzy_histogram,axis=-1)))
    sess.run(fuzzy_histogram.assign(tf.divide(fuzzy_histogram,
                                              tf.tile(tf.expand_dims(max_bin_value, -1),
                                                      [1, 1, num_bin]))))

In [16]:
image_folder = 'F:\\dataset2014\\dataset\\baseline\\highway\\input\\'

In [17]:
sess = initialize_sess()

In [18]:
for i_train in range(1,num_training_frame):
    cv_BGR_image = cv2.imread(image_folder + 'in{0:06d}'.format(i_train) + '.jpg')
    cv_gray_image = cv2.cvtColor(cv_BGR_image, cv2.COLOR_BGR2GRAY)
    cv_float_gray_image = cv_gray_image.astype('float')/256.0 # Avoid reaching the 22th bin of a histogram (not using ./255)
    cv2.imshow('cv_float_gray_image',cv_float_gray_image)
    cv2.waitKey(1)
    histogram_checking(sess, cv_float_gray_image)
    cv2.imshow('raw_segmentation',sess.run(raw_segmentation.value()))
    cv2.waitKey(1)
    temp_synthesis(sess)
    cv2.imshow('synthesis_image',sess.run(synthesis_result.value()))
    cv2.waitKey(50)
    # update_weight_calculation(sess)
    update_histogram(sess)

In [19]:
for i_train in range(num_training_frame,100):
    cv_BGR_image = cv2.imread(image_folder + 'in{0:06d}'.format(i_train) + '.jpg')
    cv_gray_image = cv2.cvtColor(cv_BGR_image, cv2.COLOR_BGR2GRAY)
    cv_float_gray_image = cv_gray_image.astype('float')/256.0 # Avoid reaching the 22th bin of a histogram (not using ./255)
    cv2.imshow('cv_float_gray_image',cv_float_gray_image)
    cv2.waitKey(1)
    histogram_checking(sess, cv_float_gray_image)
    cv2.imshow('raw_segmentation',sess.run(raw_segmentation.value()))
    cv2.waitKey(1)
    temp_synthesis(sess)
    cv2.imshow('synthesis_image',sess.run(synthesis_result.value()))
    cv2.waitKey(200)
    update_weight_calculation(sess)
    update_histogram(sess)

In [20]:
cv2.destroyAllWindows()

In [21]:
sess.close()

In [19]:
sess.run(tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),[image_height*image_width,3])).shape

(76800, 3)

In [20]:
sess.run(tf.reshape(tf.multiply(pre_weight,update_mask),[image_height*image_width,])).shape

(76800,)

In [22]:
sess.run(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                            [image_height*image_width,3]), 
                         values=tf.reshape(tf.multiply(pre_weight,update_mask),
                                           [image_height*image_width,]), 
                         dense_shape=[image_height,image_width,num_bin]))

SparseTensorValue(indices=array([[  0,   0,   5],
       [  0,   1,   1],
       [  0,   2,   2],
       ...,
       [239, 317,   7],
       [239, 318,   7],
       [239, 319,   7]], dtype=int64), values=array([0.4117651 , 0.64705884, 0.66666675, ..., 0.45098066, 0.76470613,
       0.37254906], dtype=float32), dense_shape=array([240, 320,  21], dtype=int64))

In [24]:
sess.run(tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                                                      [image_height*image_width,3]), 
                                                   values=tf.reshape(tf.multiply(pre_weight,update_mask),
                                                                     [image_height*image_width,]), 
                                                   dense_shape=[image_height,image_width,num_bin]))).shape

(240, 320, 21)

In [26]:
sess.run(fuzzy_histogram.assign_add(tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                                                                                 [image_height*image_width,3]), 
                                                                              values=tf.reshape(tf.multiply(pre_weight,update_mask),
                                                                                                [image_height*image_width,]), 
                                                                              dense_shape=[image_height,image_width,num_bin])))).shape

(240, 320, 21)

In [27]:
sess.run(fuzzy_histogram.assign_add(tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(next_index, -1)], -1)),
                                                                                                 [image_height*image_width,3]), 
                                                                              values=tf.reshape(tf.multiply(next_weight,update_mask),
                                                                                                [image_height*image_width,]), 
                                                                              dense_shape=[image_height,image_width,num_bin]))))

InvalidArgumentError: indices[4368] = [13,208,21] is out of bounds: need 0 <= index < [240,320,21]
	 [[Node: SparseToDense_6 = SparseToDense[T=DT_FLOAT, Tindices=DT_INT64, validate_indices=true, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape_18/_139, SparseTensor_8/dense_shape, Reshape_19/_141, SparseToDense_6/default_value)]]
	 [[Node: SparseToDense_6/_143 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device_incarnation=1, tensor_name="edge_22_SparseToDense_6", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"]()]]

Caused by op 'SparseToDense_6', defined at:
  File "E:\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "E:\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "E:\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel

In [28]:
sess.run(tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(next_index, -1)], -1)),[image_height*image_width,3])).shape

(76800, 3)

In [20]:
tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                                                                                     [fuzzy_histogram.shape[0]*fuzzy_histogram.shape[1],3]), 
                                                                                  values=tf.reshape(tf.multiply(pre_weight,update_mask),
                                                                                                    [fuzzy_histogram.shape[0]*fuzzy_histogram.shape[1]]), 
                                                                                  dense_shape=fuzzy_histogram.shape)).shape

TensorShape([Dimension(240), Dimension(320), Dimension(21)])

In [21]:
sess.run(tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                                                                                     [fuzzy_histogram.shape[0]*fuzzy_histogram.shape[1],3]), 
                                                                                  values=tf.reshape(tf.multiply(pre_weight,update_mask),
                                                                                                    [fuzzy_histogram.shape[0]*fuzzy_histogram.shape[1]]), 
                                                                                  dense_shape=fuzzy_histogram.shape)).shape)

TypeError: Fetch argument TensorShape([Dimension(240), Dimension(320), Dimension(21)]) has invalid type <class 'tensorflow.python.framework.tensor_shape.TensorShape'>, must be a string or Tensor. (Can not convert a TensorShape into a Tensor or Operation.)

In [28]:
tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1).shape

TensorShape([Dimension(240), Dimension(320), Dimension(3)])

In [29]:
tf.gather_nd(fuzzy_histogram,tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)).shape

TensorShape([Dimension(240), Dimension(320)])

In [30]:
tf.multiply(pre_weight,update_mask).shape

TensorShape([Dimension(240), Dimension(320)])

In [32]:
tf.add(tf.multiply(pre_weight,update_mask),
       tf.gather_nd(fuzzy_histogram,
                    tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1))).shape

TensorShape([Dimension(240), Dimension(320)])

In [ ]:
def set_value(matrix, x, y, val):
    # 得到张量的宽和高，即第一维和第二维的Size
    w = int(matrix.get_shape()[0])
    h = int(matrix.get_shape()[1])
    # 构造一个只有目标位置有值的稀疏矩阵，其值为目标值于原始值的差
    val_diff = val - matrix[x][y]
    diff_matrix = tf.sparse_tensor_to_dense(tf.SparseTensor(indices=[x, y], values=[val_diff], dense_shape=[w, h]))
    # 用 Variable.assign_add 将两个矩阵相加
    matrix.assign_add(diff_matrix)

In [35]:
tf.multiply(pre_weight,update_mask).shape

TensorShape([Dimension(240), Dimension(320)])

In [36]:
tf.sparse_tensor_to_dense(tf.SparseTensor(indices=tf.reshape(tf.to_int64(tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)),
                                                             [fuzzy_histogram.shape[0]*fuzzy_histogram.shape[1],3]), 
                                          values=tf.reshape(tf.multiply(pre_weight,update_mask),
                                                            [fuzzy_histogram.shape[0]*fuzzy_histogram.shape[1]]), 
                                          dense_shape=fuzzy_histogram.shape)).shape

TensorShape([Dimension(240), Dimension(320), Dimension(21)])

In [27]:
sess.run(tf.scatter_update(fuzzy_histogram,
                           indices=tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1),
                           updates=tf.add(tf.multiply(pre_weight,update_mask),
                                          tf.gather_nd(fuzzy_histogram,
                                                       tf.concat([indexing_constant, tf.expand_dims(pre_index, -1)], -1)
                                                      )
                                         )
                          )
        )

ValueError: Shapes must be equal rank, but are 2 and 5 for 'ScatterUpdate' (op: 'ScatterUpdate') with input shapes: [240,320,21], [240,320,3], [240,320].

In [20]:
sess.run(raw_segmentation.value())

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)

In [21]:
blurred_mask = cv2.medianBlur(sess.run(raw_segmentation.value()),9)

error: C:\Miniconda3\conda-bld\menpo_conda-opencv3_1486471199440\work\modules\imgproc\src\smooth.cpp:3552: error: (-215) src.depth() == CV_8U && (cn == 1 || cn == 3 || cn == 4) in function cv::medianBlur


In [22]:
raw_segmentation_in_np_in_float = sess.run(raw_segmentation.value())

In [23]:
raw_segmentation_in_np_in_uint8 = raw_segmentation_in_np_in_float.astype('uint8')

In [24]:
raw_segmentation_in_np_in_uint8.dtype

dtype('uint8')

In [25]:
raw_segmentation_in_np_in_float_scaled_to_255 = 255.0*sess.run(raw_segmentation.value())
raw_segmentation_in_np_in_uint8 = raw_segmentation_in_np_in_float_scaled_to_255.astype('uint8')
blurred_mask = cv2.medianBlur(raw_segmentation_in_np_in_uint8,9)

In [27]:
cv_BGR_image = cv2.imread(image_folder + 'in{0:06d}'.format(1) + '.jpg')

In [28]:
cv_gray_image = cv2.cvtColor(cv_BGR_image, cv2.COLOR_BGR2GRAY)

In [21]:
cv_gray_image.dtype

dtype('uint8')

In [28]:
cv_gray_image.dtype == 'uint8'

True

In [29]:
cv_float_gray_image = cv_gray_image.astype('float')/255

In [30]:
cv_float_gray_image.dtype

dtype('float64')

In [31]:
cv_float_gray_image[1,2]

0.11764705882352941

In [24]:
image_folder + 'in{0:06d}'.format(15) + '.jpg'

'F:\\dataset2014\\dataset\\baseline\\highway\\input\\in000015.jpg'

In [23]:
'{0:06d}'.format(13)

'000013'

In [20]:
"{0:>7d}".format(13)

'     13'

In [18]:
tf.reset_default_graph()
temp_A = tf.constant([[ 1.,  2.], [ 3. , 4.]])
temp_B = tf.tile(tf.expand_dims(temp_A, -1),[1, 1, 3])
temp_C = tf.expand_dims(tf.ones([temp_A.shape[0],1]), 1) * temp_A
with tf.Session() as sess:
    print(temp_A.eval())
    print(temp_B.eval())
    print(temp_C.eval())

[[1. 2.]
 [3. 4.]]
[[[1. 1. 1.]
  [2. 2. 2.]]

 [[3. 3. 3.]
  [4. 4. 4.]]]
[[[1. 2.]
  [3. 4.]]

 [[1. 2.]
  [3. 4.]]]


In [71]:
# Slice assignment
tf.reset_default_graph()
temp_x = tf.Variable([2.0,3.0,4.0], name='temp_x')
temp_index = [0,2]
with tf.Session() as sess:
    sess.run(tf.variables_initializer([temp_x,]))
    sess.run(tf.scatter_update(temp_x,indices=temp_index,updates=temp_index))
    print(temp_x.eval())

[0. 3. 2.]


In [67]:
temp_np_x = np.array([2,3,4])
print(temp_np_x[temp_index])

[2 4]


In [58]:
2*3**2*1

18

In [74]:
tf.reset_default_graph()
temp_x = tf.constant([2.0,3.0], name='temp_x')
with tf.Session() as sess:
    print(sess.run(tf.pow(temp_x, 5)))
    print(sess.run(temp_x*5))

[ 32. 243.]
[10. 15.]


In [36]:
temp_a = tf.Variable([1.0,2.0,3.0])

# temp_b = temp_a/2
with tf.Session() as sess:
    sess.run(tf.variables_initializer([temp_a,]))
    print(sess.run(temp_a.value()))
    print(type(sess.run(temp_a.value())))
    print(type(temp_a.eval()))
    temp_x = sess.run(temp_a.value())
    temp_T = temp_a.eval()
    sess.run(temp_a.assign([3,2,1]))
    print(temp_a.eval())

[1. 2. 3.]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[3. 2. 1.]


In [47]:
tf.reset_default_graph()
temp_x = tf.Variable(10.0, name='temp_x')
temp_y = tf.Variable(1.0, name='temp_y')
temp_z = temp_x+temp_y
temp_assign_op = temp_x.assign(temp_z)
with tf.Session() as sess:
    sess.run(tf.variables_initializer([temp_x,temp_y]))
    print(temp_x.eval())
    print(temp_y.eval())
    print(temp_z.eval())
    print(temp_assign_op.eval())
    print(temp_assign_op.eval())
    print(temp_x.eval())
    tf.summary.FileWriter('Logs', sess.graph)

10.0
1.0
11.0
11.0
12.0
12.0


In [44]:
for var in tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES ):
    print(var.name,var.shape)

temp_x:0 ()
temp_y:0 ()


In [46]:
for op in tf.get_default_graph().get_operations():
    print(op.name)

temp_x/initial_value
temp_x
temp_x/Assign
temp_x/read
temp_y/initial_value
temp_y
temp_y/Assign
temp_y/read
add
Assign
init


In [50]:
tf.reset_default_graph()
temp_x = tf.placeholder(tf.float32, name='temp_x')
temp_a = tf.Variable(0.0, name='temp_a')
temp_b = tf.Variable(0.0, name='temp_b')
temp_c = tf.Variable(0.0, name='temp_c')
temp_assign_a = temp_a.assign(temp_x+1)
temp_assign_b = temp_b.assign(temp_a+1)
temp_assign_c = temp_c.assign(temp_b+1)
with tf.Session() as sess:
    sess.run(tf.variables_initializer([temp_a,temp_b,temp_c]))
    print(temp_a.eval())
    print(temp_b.eval())
    print(temp_c.eval())
    print(temp_assign_c.eval())
    print(temp_a.eval())
    print(temp_b.eval())
    print(temp_c.eval())
    print(sess.run(temp_assign_a, feed_dict={temp_x: 7}))
    tf.summary.FileWriter('Logs', sess.graph)

0.0
0.0
0.0
1.0
0.0
0.0
1.0
8.0


In [33]:
temp_x

array([1., 2., 3.], dtype=float32)

In [37]:
temp_T

array([1., 2., 3.], dtype=float32)

In [ ]:
def temp_synthesis():
    cv2.medianBlur(raw_segmentation.value,5)